In [2]:
### This Notebook explores how to use nba_api to retrieve some important features we will be needing

In [9]:
# Imports
from nba_api.stats.static import players
from nba_api.stats.endpoints import (
    PlayerGameLog,
    BoxScorePlayerTrackV2,
    BoxScoreAdvancedV2,
    DraftCombinePlayerAnthro,
    CommonPlayerInfo
)
import pandas as pd

In [8]:
# Get the ID for the player so we can have portable lookups
pid = players.find_players_by_full_name("Stephen Curry")[0]["id"]

print(pid)
logs = PlayerGameLog(
    player_id=pid,
    season="2016-17",
    season_type_all_star="Playoffs"
).get_data_frames()[0]
# print(logs)

finals_logs = logs[logs["MATCHUP"].str.contains("CLE")]
assert not finals_logs.empty, "No Finals logs found — check filters."
print(finals_logs)

201939
  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     42016     201939  0041600405  Jun 12, 2017  GSW vs. CLE  W   41   10   
1     42016     201939  0041600404  Jun 09, 2017    GSW @ CLE  L   38    4   
2     42016     201939  0041600403  Jun 07, 2017    GSW @ CLE  W   39    8   
3     42016     201939  0041600402  Jun 04, 2017  GSW vs. CLE  W   36    7   
4     42016     201939  0041600401  Jun 01, 2017  GSW vs. CLE  W   34   11   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   20   0.500  ...     4    6   10    3    0    4   3   34           3   
1   13   0.308  ...     4    5   10    2    0    4   2   14         -24   
2   19   0.421  ...     8   13    6    2    0    1   3   26          11   
3   17   0.412  ...     8   10   11    1    0    8   3   32          21   
4   22   0.500  ...     5    6   10    3    0    2   3   28          20   

   VIDEO_AVAILABLE  
0                1  
1                1  
2         

In [5]:
game_id = finals_logs.iloc[0]["Game_ID"]
print(f"Using Finals Game_ID: {game_id}\n")

Using Finals Game_ID: 0041600405



In [6]:
# This endpoint doesn't seem to be working, its something to do with the arguments passed
# trk = BoxScorePlayerTrackV2(game_id=game_id).player_track.get_data_frame()
# spd_data = trk[trk["PLAYER_ID"] == pid][["PLAYER_NAME", "SPD", "DIST", "TCHS", "PASS"]]
# print(spd_data)

# Player stats for a given game
adv = BoxScoreAdvancedV2(game_id=game_id).player_stats.get_data_frame()
adv_data = adv[adv["PLAYER_ID"] == pid][["PLAYER_NAME", "MIN", "PACE", "USG_PCT", "TS_PCT"]]
print(adv_data)

      PLAYER_NAME    MIN   PACE  USG_PCT  TS_PCT
17  Stephen Curry  40:48  99.41    0.299   0.639


In [7]:
anthro = DraftCombinePlayerAnthro().get_data_frames()[0]
anthro_data = anthro[anthro["PLAYER_ID"] == pid][
    ["PLAYER_NAME", "HEIGHT_WO_SHOES", "HEIGHT_W_SHOES", "WEIGHT", "WINGSPAN"]
]

if anthro_data.empty:
    info = CommonPlayerInfo(player_id=pid).common_player_info.get_data_frame().iloc[0]
    anthro_data = pd.DataFrame([{
        "PLAYER_NAME": info["DISPLAY_FIRST_LAST"],
        "HEIGHT_W_SHOES": info["HEIGHT"],
        "WEIGHT": info["WEIGHT"],
        "WINGSPAN": None
    }])
print(anthro_data)

     PLAYER_NAME HEIGHT_W_SHOES WEIGHT WINGSPAN
0  Stephen Curry            6-2    185     None
